# Multi AI Agent

## Simple Multi-AI Agent Architecture

In [27]:
from typing import Annotated, TypedDict, List, Literal
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
# from typing_extensions import TypedDict

from langgraph.graph import END, START,StateGraph, MessagesState
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition, create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_groq import ChatGroq

import os
from dotenv import load_dotenv

load_dotenv()

True

In [36]:
# Defining the state
class AgentState(MessagesState):
    next_agent:str #indicates which is the next agent

# Defining sample tool
@tool
def search_web(query:str)->str:
    """Search the web for information"""
    # Using Tavily
    search = TavilySearchResults(max_results=3)
    results = search.invoke(query)
    return str(results)

@tool
def write_summary(content:str)-> str:
    """Write a summary for the provided content"""
    # Generates the summary
    summary = f"Summary of the findings:\n\n{content[:500]}..."
    return summary

# loading the groq api key and intialising the llm
groq_api_key = os.getenv("GROQ_API")
llm = ChatGroq(model = "llama-3.1-8b-instant", api_key = groq_api_key)

In [49]:
# Defining Researcher Agent
def researcher_agent(state: AgentState):
    "Researcher agent searches for the particular information"
    messages = state["messages"]

    # Adding system message for context
    system_msg = SystemMessage(content = "You are a research assistant. Use the search_web tool to find information about the user's request. Provide the URLs as sources for information")

    # LLM with tools
    researcher_llm = llm.bind_tools([search_web])
    response = researcher_llm.invoke([system_msg] + messages) 

    # Return the response and route the response to writer
    return {
        "messages": [response],
        "next_agent" : "writer"
    }

# Defining writer agent
def writer_agent(state: AgentState):
    """Writer agent that creates summaries"""
    messages = state["messages"]

    # Adding system message for context
    system_msg = SystemMessage(content = "You are a technical writer. Review the conversation and create a comprehensive, clear and concise summary of the findings.Also include the sources in the summary")

    # completion without tools
    response = llm.invoke([system_msg]+ messages)

    # Returns the response
    return{
        "messages":[response],
        "next_agent": "end"
    }

In [50]:
# Tool executor node
def execute_tools(state:AgentState):
    """Execute any pending tool calls"""
    messages = state["messages"]
    last_message = messages[-1]

    # check if there are tool calls to execute
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        # create tool node and execute
        tool_node = ToolNode([search_web, write_summary])
        response = tool_node.invoke(state)
        return response
    
    # No tools to execute
    return state

In [51]:
# Building graph
workflow = StateGraph(MessagesState)

# Adding nodes
workflow.add_node("researcher",researcher_agent)
workflow.add_node("writer", writer_agent)

# Adding edges
workflow.add_edge(START,"researcher")
workflow.add_edge("researcher","writer")
workflow.add_edge("writer",END)

final_workflow = workflow.compile()

In [52]:
# Executing final workflow
response = final_workflow.invoke({"messages":"Research about how Agentic AI can be used in the field of Agriculture"})

for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Research about how Agentic AI can be used in the field of Agriculture
================================== Ai Message ==================================
Tool Calls:
  search_web (7a1fxhgax)
 Call ID: 7a1fxhgax
  Args:
    query: Agentic AI in agriculture research
  search_web (t74mm02ge)
 Call ID: t74mm02ge
  Args:
    query: Applications of Agentic AI in agricultural sector
  search_web (1xqg9yv5g)
 Call ID: 1xqg9yv5g
  Args:
    query: Agentic AI use cases in farming
  search_web (ja6nn913e)
 Call ID: ja6nn913e
  Args:
    query: Benefits of Agentic AI in agricultural productivity
================================== Ai Message ==================================

 

**Summary:**

Agentic AI, also known as autonomous or self-aware AI, has the potential to revolutionize the field of agriculture by increasing efficiency, productivity, and sustainability. This technology can be used in various aspects of agricu

In [ ]:

# State definition
    # class SupervisorState(MessageState):
    #     """State for the supervisor multi-agent system"""
    #     next_agent: str
    #     task_assignment: Dict[str, List[str]] = {} #Tracks what each agent should do
    #     agent_outputs: Dict[str, Any] = {} #Stores output from each agent
    #     workflow_stage: str = "initial" #Tracks workflow progress
    #     iteration_count: int = 0
    #     max_iterations: int = 10
    #     final_output: str = ""

